In [ ]:
!pip install ../input/sartorius-segmentation-mask-rcnn-tf-requirements/grpcio-1.32.0-cp37-cp37m-manylinux2014_x86_64.whl
!pip install ../input/sartorius-segmentation-mask-rcnn-tf-requirements/gast-0.3.3-py2.py3-none-any.whl
!pip install ../input/sartorius-segmentation-mask-rcnn-tf-requirements/six-1.15.0-py2.py3-none-any.whl
!pip install ../input/sartorius-segmentation-mask-rcnn-tf-requirements/tensorflow_estimator-2.4.0-py2.py3-none-any.whl
!pip install ../input/sartorius-segmentation-mask-rcnn-tf-requirements/h5py-2.10.0-cp37-cp37m-manylinux1_x86_64.whl
!pip install ../input/sartorius-segmentation-mask-rcnn-tf-requirements/typing_extensions-3.7.4.3-py3-none-any.whl
!pip install ../input/sartorius-segmentation-mask-rcnn-tf-requirements/tensorflow-2.4.0-cp37-cp37m-manylinux2010_x86_64.whl
!pip install ../input/sartorius-segmentation-mask-rcnn-tf-requirements/Keras-2.4.0-py2.py3-none-any.whl
!pip install ../input/leekunhee-mask-rcnn/Mask_RCNN-master

**<h2>Training Notebook:</h2>**
https://www.kaggle.com/susnato/sartorius-segmentation-mask-rcnn-tf-train
**<h1><font color='red'>If You Find This Useful Then Please upvote</h1>**

**<h2>Imports</h2>**

In [ ]:
import os
import cv2
import shutil
import pandas as pd
import numpy as np
from numpy import zeros, asarray
import matplotlib.pyplot as plt 
%matplotlib inline

from mrcnn import utils
import mrcnn.config
import mrcnn.model as modellib
import keras
import tensorflow as tf


seed=42
np.random.seed(seed)
tf.random.set_seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

test_fol = '../input/sartorius-cell-instance-segmentation/test'
sub = pd.read_csv('../input/sartorius-cell-instance-segmentation/sample_submission.csv')
tf.__version__, keras.__version__, tf.keras.__version__

**<h2>Load Test Dataset</h2>**

In [ ]:
# Model Configuration
class KaggleSartoriusConfig(mrcnn.config.Config):
    NAME = "kaggle_Sartorius_cfg"
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1 
    NUM_CLASSES = 2
    USE_MINI_MASK = False
    
kaggle_sartorius_model_config = KaggleSartoriusConfig()

def rle_encode(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

**<h2>Load Model</h2>**

In [ ]:
# Build the Mask R-CNN Model Architecture
model = mrcnn.model.MaskRCNN(mode='inference', 
                             model_dir='./infer__mask_rcnn/', 
                             config=kaggle_sartorius_model_config)
#load trained-weights from the Train Notebook
'''
    WE ARE LOADING THE WEIGHTS OF EPOCH 5
'''
model.load_weights(filepath='../input/epochs-710/mask_rcnn_kaggle_sartorius_cfg_0010.h5', 
                   by_name=True)

**<h2>Fix Overlaps</h2>**

In [ ]:
#import cupy as cp
import gc

def fix_overlap(msk):
    """
    Args:
        mask: multi-channel mask, each channel is an instance of cell, shape:(520,704,None)
    Returns:
        multi-channel mask with non-overlapping values, shape:(520,704,None)
    """
    msk = np.array(msk)
    msk = np.pad(msk, [[0,0],[0,0],[1,0]])
    ins_len = msk.shape[-1]
    msk = np.argmax(msk,axis=-1)
    msk = tf.keras.utils.to_categorical(msk, num_classes=ins_len)
    msk = msk[...,1:]
    msk = msk[...,np.any(msk, axis=(0,1))]
    return msk

def check_overlap(msk):
    msk = msk.astype(np.bool).astype(np.uint8) # binary mask
    return np.any(np.sum(msk, axis=-1)>1) # only one channgel will contain value


**<h2>Prediction</h2>**

In [ ]:
%%time
import skimage

image_names, pred_annots = [],[]
for s in sub['id']:
    img_name, prd=[],[]
    image = skimage.io.imread(os.path.join(test_fol, s)+'.png')
    if image.ndim != 3:
        image = skimage.color.gray2rgb(image)
    if image.shape[-1] == 4:
        image = image[..., :3]
    results = model.detect([image])[0]['masks']
    print(image.shape, results.shape)
    if results.shape[-1]==0:
        img_name.append(s)
        prd.append('')
    else:
        if check_overlap(results): # if mask instances have overlap then fix it
            print("Overlap Found!")
            results = fix_overlap(results)
        for r in range(results.shape[2]):
            pred_mask = results[:, :, r]
            pred_mask = pred_mask.astype(np.uint8)
            
            img_name.append(s)
            prd.append(rle_encode(pred_mask))
        
    image_names.extend(img_name)
    pred_annots.extend(prd)


**<h2>Submission</h2>**

In [ ]:
assert len(os.listdir('../input/sartorius-cell-instance-segmentation/test'))==len(np.unique(image_names))

pd.DataFrame({'id':image_names, 'predicted':pred_annots}).sort_values(['id']).to_csv('submission.csv', index=False)
pd.read_csv('submission.csv').head(20)